## Import Libraries

In [ ]:
import FileManager
import WordCleaner
import Indexer
import Matcher
%load_ext autoreload
%autoreload 2

## Load Files

In [ ]:
dataset = FileManager.csvToDict('testing.csv')
datasets = [dataset]

## Stemming

In [ ]:
dataset = datasets[-1]
stemmed_dataset = {}
for row in dataset:
    stemmed_words = WordCleaner.stem(dataset[row], 'porter')
    stemmed_dataset[row] = stemmed_words
datasets.append(stemmed_dataset)

## OR Lemmatization

In [ ]:
dataset = datasets[-1]
lemmad_dataset = {}
for row in dataset:
    stemmed_words = WordCleaner.lemmatize(dataset[row])
    lemmad_dataset[row] = stemmed_words
datasets.append(lemmad_dataset)

## Removing Stopwords

In [ ]:
dataset = datasets[-1]
filtered_dataset = {}
for key in dataset:
    filtered_dataset[key] = WordCleaner.removeStopWords(dataset[key])
datasets.append(filtered_dataset)

## Creating the inverted index

In [ ]:
inverted_index = Indexer.getInvertedIndex(datasets[-1])
inverted_index

## Calculating tf-idf for the document

### using Scikit Learn

In [ ]:
(tfidf_matrix, df) = Indexer.calculateTF_IDF(datasets[-1])

df

### Manually

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
(tfidf_matrix, df) = Indexer.calculateManualTF_IDF(datasets[-1])

ans = csr_matrix(arg1=tfidf_matrix, dtype=np.float64)
ans
# df

## Calculate Cosine Sim

In [ ]:
related_docs = Matcher.calcCosSimWithCorpus(df,df.iloc[8])

# Print the sorted related documents
print("Related Docs (similarity > 0.5):")
for doc, sim in related_docs:
    print(f"Doc {doc}: Similarity = {sim:.4f}")

## Query Manipulation 

### Enter the Query

In [ ]:
from nltk.tokenize import word_tokenize
query = 'when was yanni born ?'
query = word_tokenize(query)

### Removing Stop Words

In [ ]:
query = WordCleaner.removeStopWords(query)
query

### Stem or Lem

In [ ]:
query = WordCleaner.stem(query, 'porter')
query

### Calculate TF-IDF

In [ ]:
(query_tfidf_matrix, qdf) = Indexer.calculateDocTF_IDF(datasets[-1],query)

qdf
# query_tfidf_matrix 

### Calculate Cos Sim

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# related_docs = Matcher.calcCosSimWithCorpus(df,query_tfidf_matrix)

A_array = np.array(qdf.iloc[0])
# A_array = np.array(df.iloc[8])
related_docs = []
for index, row in df.iterrows():
    B_array = np.array(row)  # Access the row data (vector B)
    similarity = cosine_similarity([A_array], [B_array])[0][0]
    print(f"Cosine Similarity between doc 1 and doc {index}: {similarity:.4f}")
    if similarity > 0.5:
        related_docs.append((index, similarity))
# Sort related_docs by similarity (highest to lowest)
related_docs.sort(key=lambda x: x[1], reverse=True)
# return related_docs

# Print the sorted related documents
print("Related Docs (similarity > 0.5):")
for doc, sim in related_docs:
    print(f"Doc {doc}: Similarity = {sim:.4f}")

## Write To File

In [ ]:
file_writer, file = FileManager.openCSVWriter('stemmed.csv',['id','text'])
for key in datasets[-1]:
    file_writer.writerow({'id': key, 'text': datasets[-1][key]})
file.close()